In [1]:
import pandas as pd 
import requests
import unicodedata
from tqdm import tqdm
from bs4 import BeautifulSoup

/var/folders/gl/l6pb7jfx42l02zc9950kh3fh0000gn/T/ipykernel_5589/632010323.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
wiki_data = pd.read_csv('wiki_languages_most_recent.csv')
# wiki_data

In [3]:
unesco_data = pd.read_csv('extinct_languages_links.csv')
# unesco_data

In [4]:
glottolog_data = pd.read_csv('languages_and_dialects_geo.csv')
glottolog_data

,glottocode,name,isocodes,level,macroarea,latitude,longitude
0,3adt1234,3Ad-Tekles,NaN,dialect,Africa,NaN,NaN
1,aala1237,Aalawa,NaN,dialect,Papunesia,NaN,NaN
2,aant1238,Aantantara,NaN,dialect,Papunesia,NaN,NaN
3,aari1239,Aari,aiw,language,Africa,5.95034,36.5721
4,aari1240,Aariya,aay,language,Eurasia,NaN,NaN
...,...,...,...,...,...,...,...
22106,zuwa1238,Zuwadza,NaN,dialect,Papunesia,NaN,NaN
22107,zwal1238,Zwall,NaN,dialect,Africa,NaN,NaN
22108,zyph1238,Zyphe,zyp,language,Eurasia,22.52400,93.2640
22109,zyud1238,Zyuzdin,NaN,dialect,Eurasia,NaN,NaN


In [5]:
# Glottolog is very specific in naming dialects and other varieties, so iso codes will likely be more navigable

wiki_lang = wiki_data['lang'].to_list()
unesco_lang = unesco_data['Name in English'].to_list()
wiki_unesco_lang = pd.DataFrame({'Agg': list(set(wiki_lang) & set(unesco_lang))})

glottolog_lang = pd.DataFrame({'Glotto lang': glottolog_data['name']})
print(len(set(set(glottolog_lang['Glotto lang']).intersection(set(wiki_unesco_lang['Agg'])))))


9


## Merging

In [6]:
geodata = pd.read_csv('languages_and_dialects_geo.csv')

langs = pd.read_csv('Extinct languages - DATA SUMMARY.csv')

In [7]:
geodata.head()

,glottocode,name,isocodes,level,macroarea,latitude,longitude
0,3adt1234,3Ad-Tekles,NaN,dialect,Africa,NaN,NaN
1,aala1237,Aalawa,NaN,dialect,Papunesia,NaN,NaN
2,aant1238,Aantantara,NaN,dialect,Papunesia,NaN,NaN
3,aari1239,Aari,aiw,language,Africa,5.95034,36.5721
4,aari1240,Aariya,aay,language,Eurasia,NaN,NaN


In [8]:
langs.head()

,Name in English,Number of speakers,Degree of endangerment
0,South Italian,7500000,Vulnerable
1,Sicilian,5000000,Vulnerable
2,Low Saxon,4800000,Vulnerable
3,Belarusian,4000000,Vulnerable
4,Lombard,3500000,Definitely endangered


In [9]:
len(set(geodata['name'].str.lower().values).intersection(set(langs['Name in English'].str.lower().values)))

1315

In [13]:
geodata.head()

,glottocode,name,isocodes,level,macroarea,latitude,longitude,name_normalized
0,3adt1234,3Ad-Tekles,NaN,dialect,Africa,NaN,NaN,3ad-tekles
1,aala1237,Aalawa,NaN,dialect,Papunesia,NaN,NaN,aalawa
2,aant1238,Aantantara,NaN,dialect,Papunesia,NaN,NaN,aantantara
3,aari1239,Aari,aiw,language,Africa,5.95034,36.5721,aari
4,aari1240,Aariya,aay,language,Eurasia,NaN,NaN,aariya


In [17]:
geodata['name_normalized'] = geodata['name'].str.lower()
langs['name_normalized'] = langs['Name in English'].str.lower()

merged = pd.merge(geodata, langs, left_on='name_normalized', right_on='name_normalized', how='inner')

# what does how = 'inner' do in pd.merge()


merged_drop_na = merged.dropna(subset=['isocodes'])
merged_mass = pd.merge(merged_drop_na, wiki_data, left_on = 'isocodes', right_on = 'iso3code', how = 'inner')

In [18]:
merged_mass

,glottocode,name,isocodes,level,macroarea,latitude,longitude,name_normalized,Name in English,Number of speakers,Degree of endangerment,Unnamed: 0,lang,family,dialects,iso3code,speakers,regions,off_lang,rec_min_lang
0,aasa1238,Aasax,aas,language,Africa,-4.00679,36.86480,aasax,Aasax,0,Extinct,57,Aas%C3%A1x,"['Afro-Asiatic', 'Cushitic', 'Rift', '?', 'Eas...",NaN,aas,NaN,Tanzania,NaN,NaN
1,abaz1241,Abaza,abq,language,Eurasia,44.25000,42.00000,abaza,Abaza,31000,Definitely endangered,80,Abaza_language,"['Northwest Caucasian', 'Abazgi', 'Abaza']",NaN,abq,"49,800 (2010–2014)[1]",Karachay-Cherkessia,Russia Karachay-Cherkessia,NaN
2,achu1247,Achumawi,acv,language,North America,41.09790,-121.19100,achumawi,Achumawi,10,Critically endangered,739,Achumawi_language,"['Hokan', '\xa0?\n', 'Shasta', '–Palaihnihan\x...",NaN,acv,NaN,NaN,NaN,NaN
3,akka1237,Akkala Saami,sia,language,Eurasia,67.86250,32.40460,akkala saami,Akkala Saami,0,Extinct,157,Akkala_S%C3%A1mi,"['Uralic', 'Sámi', 'Eastern', 'Mainland', 'Akk...",NaN,sia,NaN,Southwest Kola Peninsula,NaN,NaN
4,akum1238,Akum,aku,language,Africa,6.85716,9.96261,akum,Akum,600,Critically endangered,91,Akum_language,"['Niger–Congo', '?\n', 'Atlantic–Congo', 'Benu...",NaN,aku,"1,400 in Cameroon (2002)[1]few in Nigeria (no ...",Taraba State,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,warl1254,Warlpiri,wbp,language,Australia,-20.10080,131.05000,warlpiri,Warlpiri,3000,Vulnerable,287,Warlpiri_language,"['Pama–Nyungan', 'Ngumpin–Yapa', 'Ngarrkic', '...","['Warlpiri', 'Ngaliya', 'Walmala', 'Ngardilpa'...",wbp,"2,624 (2021 census)[1]","Northern Territory, Australia",NaN,NaN
107,wels1247,Welsh,cym,language,Eurasia,52.00000,-4.00000,welsh,Welsh,750000,Vulnerable,942,Welsh_language,"['Indo-European', 'Celtic', 'Insular Celtic', ...","['Cofi', 'Gwyndodeg', 'Powyseg', 'Dyfedeg', 'G...",cym,"\nWales: 538,300 (17.8% of the population of W...","United Kingdom (Wales, England), Argentina (Ch...",Wales,Argentina (Chubut Province)
108,west2392,Western Mari,mrj,language,Eurasia,56.22000,46.56900,western mari,Western Mari,36822,Severely endangered,1220,Hill_Mari,"['Uralic', 'Mari', 'Western Mari', 'Hill Mari']",NaN,mrj,"30,000 (2012)[1]","Mari El (Gornomariysky, Yurinsky, Kilemarsky d...",Russia Mari El,NaN
109,yuch1247,Yuchi,yuc,language,North America,35.75000,-86.75000,yuchi,Yuchi,5,Critically endangered,228,Yuchi_language,['Language isolate'],NaN,yuc,0[2] (2021)12 L2 speakers (2016)[1],East central Oklahoma,NaN,NaN


In [11]:
merged['level'].value_counts()

level
language    1167
dialect      153
Name: count, dtype: int64

In [12]:
merged.to_csv('merged.csv', index=False)